# Wikipedia crawler which collects all the links from given page and also check connection between all of them

Importing all the required libraries 

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import time,datetime,random
import networkx as nx
import matplotlib.pyplot as plt

urllib is used to make request to open website we want. bs4 is package which will parse the html page and store it so we can access and manipulate it. networkx is package for creating graph from given data.

In [ ]:
url = 'https://en.wikipedia.org/wiki/'
html = urlopen(url, timeout=(6.05))
bs = soup(html, 'html.parser')
html.close()
bs.find("div",{ "class":"navbox"}).decompose()
bs.find("div",{"class":"navbox authority-control"}).decompose()

We made request to wikipedia page and stored that page using bs4. Since we do not want links from navigation box in wikipedia page we just deleted those divisions using decompose.

In [ ]:
pages, title = [], []
edge_list = pd.DataFrame(columns=['V1','V2'])
for link in bs.find("div", {"id":"bodyContent"}).findAll("a",
                   href=re.compile("^(/wiki/)((?!:)(?!disambiguation)(?!International_Standard_Book_Number).)*$")):
    if link.attrs['title'] not in title:
        pages.append(link.attrs['href'])
        title.append(link.attrs['title'])

M_art = pd.DataFrame(zip(title,pages),columns=['Article','Link'])
edge_list['V2'], edge_list['V1'] = title, bs.title.text

We are checking every link on given wikipedia article and storing it only if it points to another wikipedia article. We are ignoring some unneccessary links such as ISBN, disambiguation. Our code does not store duplicate links or articles.

We are storing titles of all the articles and making edgelist out of them. Means if one article points to another article there is edge between them(directed). 

In [ ]:
for i in pages:
    url = 'https://en.wikipedia.org'+i    
    time.sleep(random.randint(2,5))
    random.seed(datetime.datetime.now())
    html = urlopen(url, timeout=(6.05))
    bs = soup(html, 'html.parser')
    html.close()
    sub_title, sub_pages = [], []
    sub_EL = pd.DataFrame(columns=['V1','V2'])
    for link in bs.find("div", {"id":"bodyContent"}).findAll("a",
                       href=re.compile("^(/wiki/)((?!:)(?!disambiguation).)*$")):	
        if link.attrs['href'] not in sub_pages:
            if link.attrs['title'] in title:
                sub_title.append(link.attrs['title'])
                sub_pages.append(link.attrs['href'])
                
    sub_EL['V2'] = sub_title
    sub_EL['V1'] = bs.title.text.replace(' - Wikipedia', '')
    M_art_sub = pd.DataFrame(zip(sub_title,sub_pages), columns=['Article','Link'])
    M_art = M_art.append(M_art_sub,  ignore_index = True)
    edge_list = edge_list.append(sub_EL, ignore_index = True)

Now we have some links from the page of our choice. Now we have to check connection between all those pages, hence we are going on each of those pages and checking if its pointing to one of its own.

We will againg get more edges from above which we will append to our original edge list

In [ ]:
G = nx.DiGraph()
for i in range(len(edge_list)):
     G.add_edge(edge_list.iloc[i,0],edge_list.iloc[i,1])
plt.figure(figsize=(8,5))
nx.draw(G, node_size=20, node_color="blue",  alpha=0.5,with_labels = False)
plt.savefig("______.svg", dpi=100)
nx.write_gml(G,'______.gml.gz')
plt.show()

We have a edgelist from which we can construct graph for visulization and calculate its statistical network properties such as degree distribution, clustering coefficient.